<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplos de ANMI

In [1]:
import numpy as np